In [19]:
import pandas as pd
import numpy as np
import itertools
import os
# Create a date range from '2000-01-01' to '2020-12-31'
date_range = pd.date_range(start='2000-01-01', end='2020-12-31', freq='D')

# Create a DataFrame with a 'date' column containing the date range
df = pd.DataFrame({'dates': date_range})
# Print the DataFrame
df.head(2)

,dates
0,2000-01-01
1,2000-01-02


In [22]:
# specify the directory path
directory_path = "/Users/rkcabhi/Downloads/GWAR Drought Data/"

# get all the files in the directory
files = os.listdir(directory_path)

# filter the files based on extension and remove extension
county_list = [os.path.splitext(file)[0] for file in files if file.endswith('.csv')]

In [25]:
# use itertools.product to generate all combinations of list and DataFrame
Dates_And_County = itertools.product(county_list, df['dates'])
# create new DataFrame with all combinations
new_df = pd.DataFrame(Dates_And_County, columns=['county', 'dates'])
new_df

,county,dates
0,San Mateo County,2000-01-01
1,San Mateo County,2000-01-02
2,San Mateo County,2000-01-03
3,San Mateo County,2000-01-04
4,San Mateo County,2000-01-05
...,...,...
391216,Glenn County,2020-12-27
391217,Glenn County,2020-12-28
391218,Glenn County,2020-12-29
391219,Glenn County,2020-12-30


In [26]:
# specify the folder path where the CSV files are located
folder_path = '/Users/rkcabhi/Downloads/GWAR Drought Data/'

# get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# create an empty list to hold the DataFrames
dfs = []

# loop through each CSV file and append its DataFrame to the list
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    dataf = pd.read_csv(file_path)
    # add a new column for the CSV file name without extension
    dataf['county'] = os.path.splitext(file)[0]
    dfs.append(dataf)

# concatenate all DataFrames into one
droughtdata = pd.concat(dfs, ignore_index=True)
droughtdata=droughtdata[droughtdata['Week']<='2020-12-31']
droughtdata.head(3)

,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI,county
120,2020-12-29,0.0,100.0,100.0,72.52,0.0,0.0,273,San Mateo County
121,2020-12-22,0.0,100.0,100.0,72.52,0.0,0.0,273,San Mateo County
122,2020-12-15,0.0,100.0,100.0,72.52,0.0,0.0,273,San Mateo County


In [27]:
# convert 'dates' and 'Week' columns to datetime objects
droughtdata['Week'] = pd.to_datetime(droughtdata['Week'])
new_df['dates'] = pd.to_datetime(new_df['dates'])
# merge the two dataframes on 'county' and 'Week'/'dates'
merged_df = pd.merge_asof(new_df.sort_values('dates'), droughtdata.sort_values('Week'), left_on='dates', right_on='Week', by='county', direction='backward')

# forward fill missing values using the last occurred metric value
merged_df[['None', 'D0-D4', 'D1-D4', 'D2-D4', 'D3-D4', 'D4', 'DSCI']] = \
    merged_df.groupby('county')[['None', 'D0-D4', 'D1-D4', 'D2-D4', 'D3-D4', 'D4', 'DSCI']].ffill()

# select the necessary columns and reset index
merged_df = merged_df[['county', 'dates', 'None', 'D0-D4', 'D1-D4', 'D2-D4', 'D3-D4', 'D4', 'DSCI']].reset_index(drop=True)

In [28]:
droughtdata[ (droughtdata['county']=='Alameda County') &  (droughtdata['Week'] >=pd.to_datetime('2016-12-06')) & (droughtdata['Week'] <= pd.to_datetime('2016-12-13'))]

,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI,county
33163,2016-12-13,0.0,100.0,100.0,82.42,44.83,0.0,327,Alameda County
33164,2016-12-06,0.0,100.0,100.0,82.42,44.83,0.0,327,Alameda County


In [29]:
merged_df[ (merged_df['county']=='Alameda County') &  (merged_df['dates'] >=pd.to_datetime('2016-12-06')) & (merged_df['dates'] < pd.to_datetime('2016-12-13'))]

,county,dates,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI
315432,Alameda County,2016-12-06,0.0,100.0,100.0,82.42,44.83,0.0,327.0
315453,Alameda County,2016-12-07,0.0,100.0,100.0,82.42,44.83,0.0,327.0
315534,Alameda County,2016-12-08,0.0,100.0,100.0,82.42,44.83,0.0,327.0
315564,Alameda County,2016-12-09,0.0,100.0,100.0,82.42,44.83,0.0,327.0
315594,Alameda County,2016-12-10,0.0,100.0,100.0,82.42,44.83,0.0,327.0
315648,Alameda County,2016-12-11,0.0,100.0,100.0,82.42,44.83,0.0,327.0
315703,Alameda County,2016-12-12,0.0,100.0,100.0,82.42,44.83,0.0,327.0


In [30]:
droughtdata[(droughtdata['county'] == 'Alpine County') &
            (pd.to_datetime(droughtdata['Week']) >= pd.to_datetime('2016-12-20')) &
            (pd.to_datetime(droughtdata['Week']) <= pd.to_datetime('2016-12-27'))]

,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI,county
29513,2016-12-27,0.0,100.0,100.0,91.31,0.00,0.0,291,Alpine County
29514,2016-12-20,0.0,100.0,100.0,91.31,1.03,0.0,292,Alpine County


In [31]:
merged_df[ (merged_df['county']=='Alpine County') &  (merged_df['dates'] >=pd.to_datetime('2016-12-20')) & (merged_df['dates'] < pd.to_datetime('2016-12-31'))]

,county,dates,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI
316134,Alpine County,2016-12-20,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316186,Alpine County,2016-12-21,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316229,Alpine County,2016-12-22,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316255,Alpine County,2016-12-23,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316333,Alpine County,2016-12-24,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316363,Alpine County,2016-12-25,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316414,Alpine County,2016-12-26,0.0,100.0,100.0,91.31,1.03,0.0,292.0
316481,Alpine County,2016-12-27,0.0,100.0,100.0,91.31,0.00,0.0,291.0
316521,Alpine County,2016-12-28,0.0,100.0,100.0,91.31,0.00,0.0,291.0
316568,Alpine County,2016-12-29,0.0,100.0,100.0,91.31,0.00,0.0,291.0


In [32]:
droughtdata[(droughtdata['county'] == 'Butte County') &
            (pd.to_datetime(droughtdata['Week']) >= pd.to_datetime('2016-12-06')) &
            (pd.to_datetime(droughtdata['Week']) <= pd.to_datetime('2016-12-13'))]

,Week,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI,county
30731,2016-12-13,0.0,100.0,31.61,0.0,0.0,0.0,132,Butte County
30732,2016-12-06,0.0,100.0,31.61,0.0,0.0,0.0,132,Butte County


In [33]:
merged_df[ (merged_df['county']=='Butte County') &  (merged_df['dates'] >=pd.to_datetime('2016-12-06')) & (merged_df['dates'] < pd.to_datetime('2016-12-13'))]

,county,dates,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI
315396,Butte County,2016-12-06,0.0,100.0,31.61,0.0,0.0,0.0,132.0
315440,Butte County,2016-12-07,0.0,100.0,31.61,0.0,0.0,0.0,132.0
315506,Butte County,2016-12-08,0.0,100.0,31.61,0.0,0.0,0.0,132.0
315584,Butte County,2016-12-09,0.0,100.0,31.61,0.0,0.0,0.0,132.0
315603,Butte County,2016-12-10,0.0,100.0,31.61,0.0,0.0,0.0,132.0
315676,Butte County,2016-12-11,0.0,100.0,31.61,0.0,0.0,0.0,132.0
315740,Butte County,2016-12-12,0.0,100.0,31.61,0.0,0.0,0.0,132.0


In [34]:
merged_df

,county,dates,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI
0,San Mateo County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Modoc County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Glenn County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Inyo County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,San Benito County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
391216,Ventura County,2020-12-31,0.0,100.0,98.75,0.00,0.00,0.0,199.0
391217,Siskiyou County,2020-12-31,0.0,100.0,100.00,100.00,88.39,0.0,388.0
391218,Mendocino County,2020-12-31,0.0,100.0,100.00,100.00,48.71,0.0,349.0
391219,Riverside County,2020-12-31,0.0,100.0,74.72,58.97,22.44,0.0,256.0


In [35]:
merged_df[merged_df.isna().any(axis=1)]

,county,dates,None,D0-D4,D1-D4,D2-D4,D3-D4,D4,DSCI
0,San Mateo County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Modoc County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Glenn County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Inyo County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,San Benito County,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
148,Mendocino County,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,San Luis Obispo County,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,Kern County,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,Fresno County,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
